# Loan Default Prediction
## Team Members:
* Harish Puvvada
* Vamsi Mohan Ramineedi

In [1]:
import os
import pandas as pd
import numpy as np
from IPython.core.display import HTML
pd.set_option("display.max_columns",75)
import warnings
warnings.filterwarnings('ignore')

In [2]:
#df2007_11 = pd.read_csv(os.getenv('FDS')+'LoanStats_2007_to_2011.csv',low_memory=False,skiprows=1)
df2012_13 = pd.read_csv(os.getenv('FDS')+'LoanStats_2012_to_2013.csv',low_memory=False,skiprows=1)
df2014 = pd.read_csv(os.getenv('FDS')+'LoanStats_2014.csv',low_memory=False,skiprows=1)

In [3]:
dataset = pd.concat([df2012_13, df2014]) #merging 2007 to 2014 datasets
dataset = dataset.iloc[:,2:111] #removing empty columns
empty_cols = [i for i in range(45,72)] #more empty columns
dataset = dataset.drop(dataset.columns[empty_cols],axis=1)
data_with_loanstatus_sliced = dataset[(dataset['loan_status']=="Fully Paid") | (dataset['loan_status']=="Charged Off")]
di = {"Fully Paid":0, "Charged Off":1} #converting target variable to boolean
Dataset_withBoolTarget= data_with_loanstatus_sliced.replace({"loan_status": di})

In [4]:
Dataset_withBoolTarget['loan_status'].value_counts()
Dataset_withBoolTarget.shape

(376233, 82)

In [5]:
#print(Dataset_withBoolTarget.shape)
dataset=Dataset_withBoolTarget.dropna(thresh = 340000,axis=1) #340000 is minimum number of non-NA values
#print(x.isnull().sum()) #- to check how many null values in all the columns
print(dataset.shape)
#dataset['loan_status'].value_counts()

(376233, 74)


In [6]:
del_col_names = ["delinq_2yrs",  "last_pymnt_d", "chargeoff_within_12_mths","delinq_amnt","emp_title", "term", "emp_title", "pymnt_plan","purpose","title", "zip_code", "verification_status", "dti","earliest_cr_line", "initial_list_status", "out_prncp",
"pymnt_plan", "num_tl_90g_dpd_24m", "num_tl_30dpd", "num_tl_120dpd_2m", "num_accts_ever_120_pd", "delinq_amnt", 
"chargeoff_within_12_mths", "total_rec_late_fee", "out_prncp_inv", "issue_d"] #deleting some more columns
dataset = dataset.drop(labels = del_col_names, axis = 1) 
dataset.shape


(376233, 52)

In [72]:
features = ['funded_amnt','emp_length','annual_inc','home_ownership','addr_state','tax_liens','grade',
            "last_pymnt_amnt", "mort_acc", "pub_rec", "int_rate", "open_acc","num_actv_rev_tl",
            "mo_sin_rcnt_rev_tl_op","mo_sin_old_rev_tl_op","bc_util","bc_open_to_buy",
            "avg_cur_bal","acc_open_past_24mths",'loan_status'] #selecting final features
Final_data = dataset[features] #19 features with target var
Final_data["int_rate"] = Final_data["int_rate"].apply(lambda x:float(x[:-1]) ) #reomving % sign, conv to float  - int_rate column
Final_data["int_rate"].corr(Final_data["loan_status"])

0.24126296486971305

In [73]:
HTML(Final_data.tail().to_html())

,funded_amnt,emp_length,annual_inc,home_ownership,addr_state,tax_liens,grade,last_pymnt_amnt,mort_acc,pub_rec,int_rate,open_acc,num_actv_rev_tl,mo_sin_rcnt_rev_tl_op,mo_sin_old_rev_tl_op,bc_util,bc_open_to_buy,avg_cur_bal,acc_open_past_24mths,loan_status
235623,15000.0,4 years,66000.0,RENT,MD,0.0,B,509.12,0.0,0.0,13.53,8.0,6.0,50.0,120.0,96.5,580.0,2877.0,1.0,0
235624,18400.0,4 years,110000.0,MORTGAGE,TX,0.0,C,7394.95,4.0,0.0,14.47,18.0,7.0,15.0,128.0,89.6,2203.0,17353.0,5.0,0
235625,22000.0,10+ years,78000.0,MORTGAGE,TN,0.0,D,17.50,5.0,1.0,19.97,18.0,5.0,17.0,198.0,54.4,5569.0,13864.0,5.0,1
235626,2000.0,3 years,83000.0,OWN,CA,0.0,A,1500.68,2.0,0.0,7.90,21.0,3.0,5.0,130.0,23.4,36402.0,28172.0,7.0,0
235628,10000.0,10+ years,46000.0,MORTGAGE,CA,0.0,D,367.20,2.0,0.0,19.20,6.0,2.0,4.0,166.0,83.9,2175.0,11495.0,4.0,0


In [74]:
'''
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
Final_data["grade"] = le.fit_transform(Final_data["grade"])
Final_data["home_ownership"] = le.fit_transform(Final_data["home_ownership"])
'''

'\nfrom sklearn import preprocessing\nle = preprocessing.LabelEncoder()\nFinal_data["grade"] = le.fit_transform(Final_data["grade"])\nFinal_data["home_ownership"] = le.fit_transform(Final_data["home_ownership"])\n'

In [75]:
Final_data['grade'] = Final_data['grade'].map({'A':7,'B':6,'C':5,'D':4,'E':3,'F':2,'G':1})
Final_data["home_ownership"] = Final_data["home_ownership"].map({"MORTGAGE":6,"RENT":5,"OWN":4,"OTHER":3,"NONE":2,"ANY":1})

In [76]:
Final_data.head(10)

,funded_amnt,emp_length,annual_inc,home_ownership,addr_state,tax_liens,grade,last_pymnt_amnt,mort_acc,pub_rec,int_rate,open_acc,num_actv_rev_tl,mo_sin_rcnt_rev_tl_op,mo_sin_old_rev_tl_op,bc_util,bc_open_to_buy,avg_cur_bal,acc_open_past_24mths,loan_status
0,27050.0,10+ years,55000.0,4,MI,0.0,6,6074.19,4.0,0.0,10.99,14.0,4.0,16.0,326.0,53.9,16473.0,9570.0,3.0,0
1,12000.0,4 years,60000.0,5,NC,0.0,6,3775.55,0.0,0.0,10.99,15.0,7.0,1.0,48.0,15.9,15216.0,476.0,8.0,0
2,4800.0,2 years,39600.0,6,TX,0.0,6,3900.48,0.0,0.0,10.99,3.0,2.0,25.0,220.0,16.1,21564.0,1379.0,0.0,0
3,12000.0,10+ years,130000.0,6,CO,0.0,6,398.28,3.0,0.0,11.99,9.0,5.0,4.0,193.0,93.0,3567.0,36362.0,4.0,0
4,7550.0,3 years,28000.0,5,CA,0.0,5,529.67,0.0,0.0,16.24,4.0,4.0,17.0,38.0,96.0,160.0,1440.0,1.0,0
5,20800.0,10+ years,81500.0,5,NY,0.0,6,13334.93,0.0,0.0,13.53,29.0,24.0,0.0,186.0,54.6,6811.0,869.0,9.0,0
6,10000.0,7 years,102000.0,6,MA,0.0,6,320.91,0.0,0.0,9.67,9.0,4.0,23.0,290.0,89.4,973.0,4349.0,3.0,0
7,27600.0,6 years,73000.0,6,CO,0.0,4,730.78,4.0,0.0,19.97,10.0,7.0,4.0,294.0,74.7,4853.0,24161.0,2.0,1
8,8000.0,2 years,33000.0,6,CO,0.0,6,276.88,0.0,1.0,10.99,9.0,5.0,14.0,269.0,72.5,2255.0,1772.0,2.0,1
9,15000.0,10+ years,98000.0,5,NY,0.0,5,14150.76,0.0,0.0,14.47,16.0,8.0,7.0,257.0,27.6,15051.0,815.0,6.0,0


In [77]:
Final_data["emp_length"].value_counts()

10+ years    122222
2 years       32828
3 years       28995
< 1 year      28121
5 years       24756
1 year        23477
4 years       21825
7 years       21701
6 years       21467
8 years       18453
n/a           17744
9 years       14644
Name: emp_length, dtype: int64

In [82]:
#Final_data["emp_length"].replace({'years','+','<':''}, regex = True)
import re
Final_data['emp_length'].replace(['years','year','+','<','n/a'], '')
Final_data['emp_length'] = re.sub(r"[^0-9]+", "", Final_data['emp_length'])


TypeError: expected string or bytes-like object

In [50]:
Final_data["emp_length"]

0          10
1           4
2           2
3          10
4           3
5          10
6           7
7           6
8           2
9          10
10          4
11          5
12          2
13         10
14         10
15          4
16          3
17          1
18         10
19         10
20         10
21         10
22          9
23         10
24          1
26         10
27          8
28          2
30         10
31          4
         ... 
235597    n/a
235598    n/a
235599      9
235600     10
235601    n/a
235602     10
235603     10
235604      4
235605      1
235606    n/a
235607     10
235608     10
235609    n/a
235610      3
235611      2
235612     10
235613      3
235614      6
235615      9
235617    n/a
235618      8
235619      7
235620      5
235621      3
235622     10
235623      4
235624      4
235625     10
235626      3
235628     10
Name: emp_length, Length: 376233, dtype: object

In [ ]:
#dataset[(dataset['avg_cur_bal'] < 5000 ) | (dataset['loan_status']==1)]

In [ ]:
features = []

In [ ]:
mandatory  - "last_pymnt_amnt", "mort_acc", "pub_rec", "int_rate", "open_acc","num_actv_rev_tl","mo_sin_rcnt_rev_tl_op","mo_sin_old_rev_tl_op","bc_util","bc_open_to_buy","avg_cur_bal","acc_open_past_24mths"

In [ ]:
doubtful - acc_now_delinq, pct_tl_nvr_dlq, num_sats, sub_grade